# Makemore - Part 1

Like previously mentioned, Makemore makes "more" of things you provide to it. Simple as that. This Jupyter notebook, is my first step in the journey in building **Makemore**. Let's do this!

Under the hood, Andrej said, Makemore is a **character-level language model**. It means that Makemore will model sequences of characters. In order words, Makemore tries to predict the next character, based on previous characters. Another way to put this, would be to say that Makemore tries to answer the following question:

> Based on the previous characters, what character is likely to come **next**?

To provide contrast, ChatGPT is a *token-level language model*. It attempts to predict the next token (i.e. words) based on the previous tokens.

Without further talking, let's start the building with loading the dataset `names.txt`.

## Loading the dataset

In this section of the notebook, I load in the dataset contained in `names.txt` in a string, split it to get individual words, then insert them in a Python list.

In [1]:
words = open('names.txt', 'r').read().splitlines()

And we can go ahead and display the first 10 element of the list. Just to see...

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']